# <center>Laboratorium Analiza i bazy danych </center>

## <center>TIER protocol i tidy data</center>

Aleksandra Stachniak, grupa 1

In [129]:
import pandas as pd
import numpy as np
import os

In [130]:
# Odczytanie ścieżki i wczytanie datasetu do pliku
path = os.getcwd()[:-13] + "original_data/weather.txt"

with open(path) as data:
    file = data.readlines()

In [131]:
# Zajmujemy się interesującym nas zakresem - pierwsze 5 miesięcy w 2010 roku
data2010 = []
for elem in file:
    if "2010" in elem[0:21]:
        month = elem[15:17]
        if month == '01' or month == '02' or month == '03' or month == '04' or month == '05':
            data2010.append(elem)

In [132]:
col_names = ["id", "year", "month", "element"] + ["day{}".format(i) for i in range(1, 32)]
stationID = data2010[0][0:11]

generate_rows = []

for row in data2010:
    it = len(stationID)
    year = row[it:it+4]
    month = row[it+4:it+4+2]
    element_type = row[it+4+2:it+4+2+4]
    days_1 = row[it+4+2+4:].split()
    day_data = row[it+4+2+4:]
    
    # Ograniczenie próbek
    if len(days_1) > 31:
        days_1 = days_1[:31]

    to_return = []
    for el in days_1:
        # Usuwanie niepoprawnych danych
        if el == 'S-9999' or el == 'S' or el == '-9999':
            el = 'inf'
            to_return.append(el)
        else:
            to_return.append(el)
    
    generate_rows.append([stationID, year, month, element_type] + to_return)

In [133]:
df = pd.DataFrame(generate_rows, columns=col_names)
df.head(10)

,id,year,month,element,day1,day2,day3,day4,day5,day6,...,day22,day23,day24,day25,day26,day27,day28,day29,day30,day31
0,MX000017004,2010,01,TMAX,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,278,inf
1,MX000017004,2010,01,TMIN,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,145,inf
2,MX000017004,2010,01,PRCP,inf,inf,inf,inf,0,inf,...,inf,0,inf,0,inf,0,inf,0,inf,0
3,MX000017004,2010,02,TMAX,inf,273,inf,241,inf,inf,...,inf,inf,299,inf,inf,inf,inf,inf,inf,inf
4,MX000017004,2010,02,TMIN,inf,144,inf,144,inf,inf,...,inf,inf,107,inf,inf,inf,inf,inf,inf,inf
5,MX000017004,2010,02,PRCP,inf,0,inf,0,inf,inf,...,0,inf,inf,0,inf,0,inf,inf,inf,inf
6,MX000017004,2010,03,TMAX,inf,inf,inf,inf,321,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
7,MX000017004,2010,03,TMIN,inf,inf,inf,inf,142,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
8,MX000017004,2010,03,PRCP,inf,0,inf,0,inf,0,...,0,inf,0,inf,0,inf,0,inf,inf,0
9,MX000017004,2010,04,TMAX,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,363,inf,inf,inf,inf


In [134]:
df_1 = pd.melt(df, ['id', 'year', 'month', 'element'])
df_1 = df_1[df_1.value != 'inf']
df_1 = df_1.replace(to_replace=["day"],value="",regex=True)

df_1.head(10)

,id,year,month,element,variable,value
11,MX000017004,2010,04,PRCP,1,0
18,MX000017004,2010,02,TMAX,2,273
19,MX000017004,2010,02,TMIN,2,144
20,MX000017004,2010,02,PRCP,2,0
23,MX000017004,2010,03,PRCP,2,0
41,MX000017004,2010,04,PRCP,3,0
48,MX000017004,2010,02,TMAX,4,241
49,MX000017004,2010,02,TMIN,4,144
50,MX000017004,2010,02,PRCP,4,0
53,MX000017004,2010,03,PRCP,4,0


In [138]:
# Aby zostały spełnione zasady tidy data łączę kolumny zawierające informacje o roku, miesiącu i dniu w jedną kolumnę zawierającą datę
df_2 = df_1.assign(date = df_1.year.astype(str) + "-" + df_1.month.astype(str) + "-" + df_1.variable.astype(str)[:] )
df_2.pop("year")
df_2.pop("variable")
df_2.pop("month")
df_2 = df_2[["id","date","element","value"]]

df_2.head(10)

,id,date,element,value
11,MX000017004,2010-04-1,PRCP,0
18,MX000017004,2010-02-2,TMAX,273
19,MX000017004,2010-02-2,TMIN,144
20,MX000017004,2010-02-2,PRCP,0
23,MX000017004,2010-03-2,PRCP,0
41,MX000017004,2010-04-3,PRCP,0
48,MX000017004,2010-02-4,TMAX,241
49,MX000017004,2010-02-4,TMIN,144
50,MX000017004,2010-02-4,PRCP,0
53,MX000017004,2010-03-4,PRCP,0


In [136]:
path = os.getcwd()[:-13] + "analysis_data/data.txt"
with open(path, 'w') as file:
    file.write(df_2.to_csv(index=False))

## Wnioski:
Ćwiczenia pozwoliły na zapoznanie się z protokołem TIER oraz na poznanie pojęcia "Tidy Data". Dzięki temu miałam okazję zastosować zasady obowiązujące i czynności, które należy wykonywać przy porządkowaniu danych.